# Домашнее задание 2 

## Глубинное обучение в анализе графовых данных, ПМИ ВШЭ


В этом дз для мониторинга экспериментов лучше использовать сервис __wandb__, [здесь](https://docs.wandb.ai/quickstart) можете ознакомиться с документацией

### 1. PageRank Personalized (1 балл)

Реализуйте на основе кода с семинара персонализованный алгоритм PageRank с опцией возвращения в одну точку (индекс который подается в keep_updating_until_convergence(...)), либо же topic-related с возможностью вернуться в определенный массив точек (подается в эту же функцию). Подумайте над реализацией, опишите как вы решили модифицирвоать алгоритм, продемонстрируйте работспособность на каком-нибудь искуственном наборе данных (можно двудольном). 

### Реализация слоев графовых нейронных сетей для классификации вершин

На 5ом семинаре мы реализовали слой **GraphSAGE** ([Hamilton et al. (2017)](https://arxiv.org/abs/1706.02216)). Здесь вы должны реализовать еще более мощные слои: **GAT** ([Veličković et al. (2018)](https://arxiv.org/abs/1710.10903)) и **GCN** ([Kipf et al. (2017)](https://arxiv.org/abs/1609.02907)). Затем вы должны запустить модели для решения задачи классификации вершин на наборе данных CORA.

In [ ]:
# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
# !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
# !pip install torch-geometric

Ниже приведен общий модуль GNN, куда можно добавить любой реализованный слой GNN

In [ ]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax


class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args, emb=False):
        super(GNNStack, self).__init__()
        conv_model = self.build_conv_model(args.model_type)
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(args.heads * hidden_dim, hidden_dim))

        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(args.heads * hidden_dim, hidden_dim), nn.Dropout(args.dropout), 
            nn.Linear(hidden_dim, output_dim))

        self.dropout = args.dropout
        self.num_layers = args.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            # When applying GAT with num heads > 1, you need to modify the 
            # input and output dimension of the conv layers (self.convs),
            # to ensure that the input dim of the next layer is num heads
            # multiplied by the output dim of the previous layer.
            # HINT: In case you want to play with multiheads, you need to change the for-loop that builds up self.convs to be
            # self.convs.append(conv_model(hidden_dim * num_heads, hidden_dim)), 
            # and also the first nn.Linear(hidden_dim * num_heads, hidden_dim) in post-message-passing.
            return GAT

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
          
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout,training=self.training)

        x = self.post_mp(x)

        if self.emb == True:
            return x

        return F.log_softmax(x, dim=1)

    def loss(self, pred, label):
        return F.nll_loss(pred, label)


### GCN (4 балла)

Слой GCN математически определяется как
$$\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{\deg(j)}} \cdot \left( \mathbf{W}^{\top} \cdot \mathbf{x}_j^{(k-1)} \right) + \mathbf{b},$$

где признаки соседних узлов сначала преобразуются матрицей весов $W$, нормализуются по их степени и суммируются. Наконец, мы применяем вектор смещения $b$ к агрегированному результату. Эту формулу можно разделить на следующие шаги:

1. Добавить петли в матрицу смежности (можно использовать функцию add_self_loops из torch_geometric).
2. Линейное преобразование матрицу призанков вершин.
3. Вычисление коэффициентов нормализации.
4. Нормирование призанаков вершин.
5. Суммирование признаков соседних вершин (агрегация «add»).
6. Применение вектора смещения.

Шаги 1-3 обычно вычисляются до передачи сообщения. Шаги 4-5 можно легко выполнить с помощью базового класса MessagePassing. Ниже вам нужно реализовать методы *message* и *forward* на основе message passing 

In [ ]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # TODO: Your code here! 
        pass


    def message(self, x_j, norm):
        # TODO: Your code here! 
        pass

### GAT (6 баллов)

Ниже вам нужно реализовать методы *message*, *forward* и *aggregate* для Graph Attention Network слоя на основе message passing

In [ ]:
class GAT(MessagePassing):

    def __init__(self, in_channels, out_channels, heads = 2,
                 negative_slope = 0.2, dropout = 0., **kwargs):
        super(GAT, self).__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.lin_l = None
        self.lin_r = None
        self.att_l = None
        self.att_r = None

        # TODO: Your code here!
        
        # Define the layers needed for the message functions
        # self.lin_l is the linear transformation that you apply to embeddings 
        # before message passing

        self.lin_r = self.lin_l

        # TODO: Your code here! 

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.lin_l.weight)
        nn.init.xavier_uniform_(self.lin_r.weight)
        nn.init.xavier_uniform_(self.att_l)
        nn.init.xavier_uniform_(self.att_r)

    def forward(self, x, edge_index, size = None):
        H, C = self.heads, self.out_channels
        # TODO: Your code here! 
        pass


    def message(self, x_j, alpha_j, alpha_i, index, ptr, size_i):
        # TODO: Your code here! 
        pass



    def aggregate(self, inputs, index, dim_size = None):
        # TODO: Your code here! 
        pass

Ниже код для тестирования GNN слоев

In [ ]:
import torch.optim as optim


def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler is None:
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [ ]:
import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args):
    print("Node task. test set size:", np.sum(dataset[0]['test_mask'].numpy()))
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

    # build model
    model = GNNStack(dataset.num_node_features, args.hidden_dim, dataset.num_classes, 
                     args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        for batch in loader:
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            pred = pred[batch.train_mask]
            label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        losses.append(total_loss)

        if epoch % 10 == 0:
            test_acc = test(test_loader, model)
            test_accs.append(test_acc)
            if test_acc > best_acc:
                best_acc = test_acc
                best_model = copy.deepcopy(model)
        else:
            test_accs.append(test_accs[-1])
    
    return test_accs, losses, best_model, best_acc, test_loader

def test(loader, test_model, is_validation=False, save_model_preds=False, model_type=None):
    test_model.eval()

    correct = 0
    # Note that Cora is only one graph!
    for data in loader:
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            pred = test_model(data).max(dim=1)[1]
            label = data.y

        mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = label[mask]

        if save_model_preds:
            print ("Saving Model Predictions for Model Type", model_type)

            data = {}
            data['pred'] = pred.view(-1).cpu().detach().numpy()
            data['label'] = label.view(-1).cpu().detach().numpy()

        correct += pred.eq(label).sum().item()

    total = 0
    for data in loader.dataset:
        total += torch.sum(data.val_mask if is_validation else data.test_mask).item()

    return correct / total
  
class ObjectView(object):
    def __init__(self, d):
        self.__dict__ = d


In [ ]:
args = ObjectView({
    'model_type': 'GraphSage',
    'dataset': 'cora',
    'num_layers': 2,
    'heads': 1,
    'batch_size': 32,
    'hidden_dim': 32,
    'dropout': 0.3,
    'epochs': 500,
    'opt': 'adam',
    'opt_scheduler': None,
    'opt_restart': 0,
    'weight_decay': 5e-3,
    'lr': 0.01
})

Настройте аргументы и запустите функции train и test на разных блоках (*GraphSAGE* с семинара, *GCN*, *GAT*) на датасете Cora и на любом другом на выбор из доступных в библиотеке torch_geometric и сравните полученное качество на разных слоях.

Также поэксперементируйте с параметрами сетей и на основе экспериментов ниже напишите выводы. 

__Отчет по экспериментам:__

### Бонус (1 балл)
* Прикрепить ссылку на отчет по экспериментам из wandb (1 балл)